In [7]:
#global variables 

NUM_LAYERS =  2
ROWS0, COLS0 = 32, 4
ROWS1, COLS1 = 32, 32

FIFO_DEPTH = 12 #aka number of vectors 
NUM_FEATURES = COLS0

rowsize = [ROWS0, ROWS1]
colsize = [COLS0, COLS1]
MAX_ROWS = max(rowsize)
MAX_COLS = max(colsize)
MAX_NODES = MAX_ROWS
LAST_ROW = rowsize[-1]

In [8]:
from pynq import Overlay, allocate, PL
import numpy as np
import time

#PL.reset()
ol = Overlay("design_5.bit")
ip = ol.algo_main_0

In [9]:
ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  input_data_1 = Register(input_data=write-only),
  input_data_2 = Register(input_data=write-only),
  user_control = Register(user_control=write-only, RESERVED=write-only),
  layer_num = Register(layer_num=write-only, RESERVED=write-only),
  weight_rows = Register(weight_rows=write-only, RESERVED=write-only),
  weight_cols = Register(weight_cols=write-only, RESERVED=write-only),
  busy = Register(busy=0, RESERVED=0),
  busy_ctrl = Register(busy_ap_vld=0, RESERVED=0),
  read_row = Register(read_row=write-only, RESERVED=write-only),
  read_col = Register(read_col=write-only, RESERVED=write-only),
  read_data = Register(read_data=0, RESERVED=0),
  read_data_ctrl = Regis

In [10]:
# Allocate contiguous memory for matrix data
w1 = allocate(shape=(ROWS0 * COLS0,), dtype=np.int32) #ensure dtype is consistent with the Vitis data.h 

for row in range(ROWS0):
    for col in range(COLS0):
        w1[row * COLS0 + col] = (row + col)*(2*(row%2)-1) 
        
w1.flush() 

# Write matrix base address to the IP (split into two 32-bit words)
ip.register_map.input_data_1 = w1.device_address & 0xFFFFFFFF
ip.register_map.input_data_2 = (w1.device_address >> 32) & 0xFFFFFFFF 

print("Weights tensor: ", w1)

Weights tensor:  [  0  -1  -2  -3   1   2   3   4  -2  -3  -4  -5   3   4   5   6  -4  -5
  -6  -7   5   6   7   8  -6  -7  -8  -9   7   8   9  10  -8  -9 -10 -11
   9  10  11  12 -10 -11 -12 -13  11  12  13  14 -12 -13 -14 -15  13  14
  15  16 -14 -15 -16 -17  15  16  17  18 -16 -17 -18 -19  17  18  19  20
 -18 -19 -20 -21  19  20  21  22 -20 -21 -22 -23  21  22  23  24 -22 -23
 -24 -25  23  24  25  26 -24 -25 -26 -27  25  26  27  28 -26 -27 -28 -29
  27  28  29  30 -28 -29 -30 -31  29  30  31  32 -30 -31 -32 -33  31  32
  33  34]


In [11]:
#run LOAD_MATRIX - these are your "weights"
print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)

# Trigger BRAM load
ip.register_map.user_control = 1
ip.register_map.layer_num = 0 
ip.register_map.weight_rows = ROWS0
ip.register_map.weight_cols = COLS0

ip.register_map.CTRL.AP_START = 0
ip.register_map.CTRL.AP_START = 1

print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)
print("nstates_visited", ip.register_map.nstates_visited_out)
print("busy =", ip.register_map.busy)

CTRL.AP_DONE = 0
CTRL.AP_IDLE = 1
CTRL.AP_START = 0
CTRL.AP_DONE = 1
CTRL.AP_IDLE = 1
CTRL.AP_START = 0
nstates_visited 0x3
busy = 0x0


In [12]:
# Allocate contiguous memory for matrix data
w2 = allocate(shape=(ROWS1 * COLS1,), dtype=np.int32) 

for row in range(ROWS1):
    for col in range(COLS1):
        w2[row * COLS1 + col] = (row + col)*(2*(row%2)-1) 
        
w2.flush() 

# Write matrix base address to the IP (split into two 32-bit words)
ip.register_map.input_data_1 = w2.device_address & 0xFFFFFFFF
ip.register_map.input_data_2 = (w2.device_address >> 32) & 0xFFFFFFFF 

print("Weights tensor: ", w2)

Weights tensor:  [ 0 -1 -2 ... 60 61 62]


In [13]:
#run LOAD_MATRIX - these are your "weights"
print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)

# Trigger BRAM load
ip.register_map.user_control = 1
ip.register_map.layer_num = 1 
ip.register_map.weight_rows = ROWS1
ip.register_map.weight_cols = COLS1

ip.register_map.CTRL.AP_START = 0
ip.register_map.CTRL.AP_START = 1

print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)
print("nstates_visited", ip.register_map.nstates_visited_out)
print("busy =", ip.register_map.busy)

CTRL.AP_DONE = 0
CTRL.AP_IDLE = 1
CTRL.AP_START = 0
CTRL.AP_DONE = 1
CTRL.AP_IDLE = 1
CTRL.AP_START = 0
nstates_visited 0x6
busy = 0x0


In [14]:
#read weights matrices 

row = 12
col = 1

for i in range(NUM_LAYERS): 
    
    ip.register_map.layer_num = i
    ip.register_map.read_row = row
    ip.register_map.read_col = col
    
    ip.register_map.user_control = 5
    ip.register_map.CTRL.AP_START = 1
    
    value = ip.register_map.read_data
    expected = (row + col)*(2*(row%2)-1) 

    print(f"Read WEIGHTS[{i}][{row}][{col}] = {value} (expected {expected})")

    ip.register_map.CTRL.AP_START = 0

Read WEIGHTS[0][12][1] = 0xfffffff3 (expected -13)
Read WEIGHTS[1][12][1] = 0xfffffff3 (expected -13)


In [15]:
#Then using the same "input_data" pointer, create a new tensor holding 'input vectors' for your fifo:
inputs = allocate(shape=(FIFO_DEPTH * NUM_FEATURES,), dtype=np.int32)

#Fill with test pattern
for row in range(FIFO_DEPTH): 
    for col in range(NUM_FEATURES): 
        inputs[row * NUM_FEATURES + col] =(row + col)*(2*(row%2)-1) 

inputs.flush()

#Write matrix base address to the IP (split into two 32-bit words)
ip.register_map.input_data_1 = inputs.device_address & 0xFFFFFFFF 
ip.register_map.input_data_2 = (inputs.device_address >> 32) & 0xFFFFFFFF

print("Fifo of input vectors: ", inputs)

Fifo of input vectors:  [  0  -1  -2  -3   1   2   3   4  -2  -3  -4  -5   3   4   5   6  -4  -5
  -6  -7   5   6   7   8  -6  -7  -8  -9   7   8   9  10  -8  -9 -10 -11
   9  10  11  12 -10 -11 -12 -13  11  12  13  14]


In [16]:
#run LOAD_FIFO - these are your input vectors

print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)

# Trigger BRAM load
ip.register_map.user_control = 2
ip.register_map.CTRL.AP_START = 0
ip.register_map.CTRL.AP_START = 1

print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)
print("nstates_visited", ip.register_map.nstates_visited_out)
print("busy =", ip.register_map.busy)

CTRL.AP_DONE = 0
CTRL.AP_IDLE = 1
CTRL.AP_START = 0
CTRL.AP_DONE = 1
CTRL.AP_IDLE = 1
CTRL.AP_START = 0
nstates_visited 0xf
busy = 0x0


In [17]:
#verify that the fifo was loaded correctly 
#first and second row loads as expected
#actually I think the whole thing loads as expected. 

row = 2
col = 2

ip.register_map.read_row = row
ip.register_map.read_col = col
ip.register_map.user_control = 6
ip.register_map.CTRL.AP_START = 1

print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("done_ctrl.done_ap_vld =", ip.register_map.done_ctrl.done_ap_vld)

value = ip.register_map.read_data
expected = (row + col)*(2*(row%2)-1)
print(f"Read VEC_STORAGE[{row}][{col}] = {value} (expected {expected})")

ip.register_map.CTRL.AP_START = 0  # Clear start for next cycle

CTRL.AP_DONE = 1
done_ctrl.done_ap_vld = 1
Read VEC_STORAGE[2][2] = 0xfffffffc (expected -4)


In [18]:
#Then using the same "input_data" pointer, create a new tensor holding the biases. 
biases = allocate(shape=(MAX_NODES * NUM_LAYERS,), dtype=np.int32)

#Fill with test pattern
for row in range(NUM_LAYERS): 
    for col in range(MAX_NODES): 
        biases[row * MAX_NODES + col] =(row + col)*(2*(row%2)-1) 

biases.flush()

#Write matrix base address to the IP (split into two 32-bit words)
ip.register_map.input_data_1 = biases.device_address & 0xFFFFFFFF 
ip.register_map.input_data_2 = (biases.device_address >> 32) & 0xFFFFFFFF

print("Bias tensor: ", biases)

Bias tensor:  [  0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14 -15 -16 -17
 -18 -19 -20 -21 -22 -23 -24 -25 -26 -27 -28 -29 -30 -31   1   2   3   4
   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  24  25  26  27  28  29  30  31  32]


In [19]:
#run LOAD_BIAS 

print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)

# Trigger BRAM load
for i in range(NUM_LAYERS): 
    
    ip.register_map.user_control = 3
    ip.register_map.layer_num = i 
    ip.register_map.weight_rows = rowsize[i] 
    ip.register_map.CTRL.AP_START = 0
    ip.register_map.CTRL.AP_START = 1
    

print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)
print("nstates_visited", ip.register_map.nstates_visited_out)
print("busy =", ip.register_map.busy)

CTRL.AP_DONE = 0
CTRL.AP_IDLE = 1
CTRL.AP_START = 0
CTRL.AP_DONE = 1
CTRL.AP_IDLE = 1
CTRL.AP_START = 0
nstates_visited 0x18
busy = 0x0


In [20]:
#verify that the biases were loaded correctly 

row = 0
col = 10

ip.register_map.read_row = row
ip.register_map.read_col = col
ip.register_map.user_control = 7
ip.register_map.CTRL.AP_START = 1

print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("done_ctrl.done_ap_vld =", ip.register_map.done_ctrl.done_ap_vld)

value = ip.register_map.read_data
expected = (row + col)*(2*(row%2)-1)

print(f"Read BIASES[{row}][{col}] = {value} (expected {expected})")

ip.register_map.CTRL.AP_START = 0  # Clear start for next cycle

CTRL.AP_DONE = 1
done_ctrl.done_ap_vld = 1
Read BIASES[0][10] = 0xfffffff6 (expected -10)


In [21]:
#Run the COMPUTE stage (this has been edited to compute and store all the matmul results in one run.) 

ip.register_map.user_control = 4
ip.register_map.CTRL.AP_START = 0
ip.register_map.CTRL.AP_START = 1

          
print("Total number of states visited so far: ", ip.register_map.nstates_visited_out)
print("CTRL.AP_DONE =", ip.register_map.CTRL.AP_DONE)
print("CTRL.AP_IDLE =", ip.register_map.CTRL.AP_IDLE)
print("CTRL.AP_START =", ip.register_map.CTRL.AP_START)

Total number of states visited so far:  0x1e
CTRL.AP_DONE = 1
CTRL.AP_IDLE = 1
CTRL.AP_START = 0


In [22]:
#check the results for vector 1 only. 

vec=0 
    
print(f"Results for vector {vec}:")

for idx in range(LAST_ROW): 

    ip.register_map.read_col = idx
    ip.register_map.read_row = vec
    ip.register_map.user_control = 8
    ip.register_map.CTRL.AP_START = 1 

    print(ip.register_map.read_data)
    ip.register_map.CTRL.AP_START = 0

print("\nTotal number of states visited so far: ", ip.register_map.nstates_visited_out)
print("-------\n")

Results for vector 0:
0x0
0x7392
0x0
0x7eb4
0x0
0x89d6
0x0
0x94f8
0x0
0xa01a
0x0
0xab3c
0x0
0xb65e
0x0
0xc180
0x0
0xcca2
0x0
0xd7c4
0x0
0xe2e6
0x0
0xee08
0x0
0xf92a
0x0
0x1044c
0x0
0x10f6e
0x0
0x11a90

Total number of states visited so far:  0x7e
-------



In [ ]:
for vec in range(FIFO_DEPTH): 
    
    print(f"Results for vector {vec}:", end=" ")
    
    for idx in range(LAST_ROW): 

        ip.register_map.read_col = idx
        ip.register_map.read_row = vec
        ip.register_map.user_control = 8
        ip.register_map.CTRL.AP_START = 1 
        
        print(ip.register_map.read_data, end=" ")
        ip.register_map.CTRL.AP_START = 0

    print("\nTotal number of states visited so far: ", ip.register_map.nstates_visited_out)
    print("-------\n")